In [125]:
!pip install tensorflow tensorflow~gpu pandas matplotlib sklearn

ERROR: Invalid requirement: 'tensorflow~gpu'


In [126]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [127]:
df=pd.read_csv('train.csv/train.csv')


In [128]:
from tensorflow.keras.layers import TextVectorization


In [129]:
X=df[['comment_text']]
Y=df[df.columns[2:]].values
MAX_FEATURES=200000

In [130]:
vector=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')


In [131]:
vector.adapt(X.values)
vector.get_vocabulary()
vector_txt=vector(X.values)

In [ ]:
ds=tf.data.Dataset.from_tensor_slices((vector_txt,Y))
ds=ds.cache()
ds=ds.shuffle(16)
ds=ds.batch(16)
ds=ds.prefetch(8)

In [ ]:
train=ds.take(int(len(ds)*.7))
val=ds.skip(int(len(ds)*.7)).take(int(len(ds)*.2))
test=ds.skip(int(len(ds)*.9)).take(int(len(ds)*.1))

In [ ]:
from tensorflow.keras.layers import LSTM,Dropout,Bidirectional,Dense,Embedding
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()


In [ ]:
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))
model.compile(loss='BinaryCrossentropy',optimizer='Adam')


In [1]:
model.fit(train,epochs=10,validation_data=val)

NameError: name 'model' is not defined

In [55]:
model.save('comments_toxic.h5')

In [116]:
from tensorflow.keras.metrics import Recall
from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import CategoricalAccuracy

In [117]:
pre=Precision()
re=Recall()
acc=CategoricalAccuracy()
for batch in test.as_numpy_iterator():
    X_t,y_t=batch
    yhat=model.predict(X_t)
    y_t=y_t.flatten()
    yhat=yhat.flatten()
    pre.update_state(y_t,yhat)
    re.update_state(y_t,yhat)
    acc.update_state(y_t,yhat)
print(f"P : {pre.result().numpy()}")
print(f"Re : {re.result().numpy()}")
print(f"acc : {acc.result().numpy()}")

1/1 [==============================] - 0s 88ms/step
P : 0.3333333432674408
Re : 0.15789473056793213
acc : 0.3333333432674408


In [124]:
import tensorflow as tf
import gradio as gr
from tensorflow.keras.layers import TextVectorization
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')
#model=tf.keras.models.load_model('comments_toxic.h5')
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(np.expand_dims(vectorized_comment,0))
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')
interface.launch(share=True)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


Running on local URL:  http://127.0.0.1:7870

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app. 

Also please ensure that your antivirus or firewall is not blocking the binary file located at: c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\frpc_windows_amd64_v0.2


In [122]:
input_str = vector('i love you')
res = model.predict(np.expand_dims(input_str,0))
res

1/1 [==============================] - 0s 61ms/step


array([[0.69943994, 0.00090404, 0.03091706, 0.00472472, 0.03595812,
        0.00086463]], dtype=float32)